## Extração de características

Nesta etapa, realizamos o feature extraction utilizando a arquitetura VGG16, uma rede neural convolucional amplamente reconhecida no campo de reconhecimento de imagem (66). No entanto, adaptamos a VGG16 para atender às necessidades específicas deste projeto.

Removemos as camadas finais de classificação da VGG16, uma vez que nosso foco está na extração de características e não na classificação.

#### Importando as bibliotecas utilizadas

In [ ]:
import ssl
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image as tf_img
import json
import numpy as np
import os

ssl._create_default_https_context = ssl._create_unverified_context

#### Instanciando a VGG16 e configurando-a

In [2]:
#Retirando as camadas da VGG que fazer a classificação, deixando somente as que fazem a extração de features
model = VGG16(weights='imagenet')
model = Model(inputs=model.inputs, outputs=model.layers[-4].output)

2023-05-20 18:24:54.084995: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 18:24:59.941677: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
# Setando o path da pasta com as imagens para a base de dados
dataset_dir = './predict_images'
images_list = os.listdir(path=dataset_dir)

In [23]:
def extract_feature(image):

    #Coloando as imagens no formato esperado pela VGG
    img = load_img(f"{dataset_dir}/{image}", target_size = (224,224))
    img = img.resize((224,224))
    img_data = tf_img.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    #O predict que vai fazer a extração de características, já que retiramos as camadas da rede que fazem a classificação
    vgg16_feature = model.predict(img_data, verbose=2)

    image_feature = {
      'image_name': image,
      'feature': vgg16_feature.tolist()
    }

    #Salvo as features e a sua classe numa lista
    return image_feature

In [24]:
import concurrent.futures

# Criar uma lista vazia para armazenar os dicionários
features_list = []

# Percorrendo as imagens
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Submeta cada imagem para o processamento
    futures = [executor.submit(extract_feature, img) for img in images_list]

    # Espere até que todas as tarefas sejam concluídas
    for future in concurrent.futures.as_completed(futures):
        try:
            # Obtenha o resultado do futuro e estenda a lista de recursos
            image_features = future.result()
            features_list.append(image_features)
        except Exception as e:
            print(f"Erro ao processar imagem: {e}")


1/1 - 0s - 117ms/epoch - 117ms/step
1/1 - 0s - 145ms/epoch - 145ms/step
1/1 - 0s - 498ms/epoch - 498ms/step
1/1 - 0s - 491ms/epoch - 491ms/step
1/1 - 1s - 545ms/epoch - 545ms/step
1/1 - 1s - 528ms/epoch - 528ms/step
1/1 - 1s - 546ms/epoch - 546ms/step
1/1 - 1s - 521ms/epoch - 521ms/step
1/1 - 0s - 390ms/epoch - 390ms/step
1/1 - 0s - 368ms/epoch - 368ms/step
1/1 - 0s - 130ms/epoch - 130ms/step
1/1 - 0s - 127ms/epoch - 127ms/step
1/1 - 0s - 127ms/epoch - 127ms/step
1/1 - 0s - 108ms/epoch - 108ms/step
1/1 - 0s - 175ms/epoch - 175ms/step
1/1 - 0s - 487ms/epoch - 487ms/step
1/1 - 1s - 506ms/epoch - 506ms/step
1/1 - 0s - 491ms/epoch - 491ms/step
1/1 - 1s - 533ms/epoch - 533ms/step
1/1 - 0s - 466ms/epoch - 466ms/step
1/1 - 0s - 472ms/epoch - 472ms/step
1/1 - 0s - 397ms/epoch - 397ms/step
1/1 - 0s - 354ms/epoch - 354ms/step
1/1 - 0s - 159ms/epoch - 159ms/step
1/1 - 0s - 176ms/epoch - 176ms/step
1/1 - 0s - 290ms/epoch - 290ms/step
1/1 - 0s - 300ms/epoch - 300ms/step
1/1 - 1s - 520ms/epoch - 520

In [25]:
#Salvo a extração em um arquivo json para não precisar extrair toda hora, já que é um processo custoso e demorado.
#Assim, posso apenas fazer um load nos códigos que implementam os classificadores

with open('./database/feature_extraction.json', 'wb') as file:
    file.write(json.dumps(features_list).encode('utf-8'))